In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [2]:
import senteval
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, models
from glob import glob

In [3]:
# SentEval prepare and batcher
def prepare(params, samples):
    return

def batcher(params, batch):
    batch = [' '.join(sent) if sent != [] else '.' for sent in batch]
    embeddings = params['encoder'](batch)
    return embeddings

# Parameter check

In [3]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: 
            continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
#     print(table)
    print(f"Total Trainable Params: {total_params:,}")
    return total_params

pool_mode='cls'
mlp_mode=True
model_list = glob(f'output/Simcse_original_*{pool_mode}*MLP{mlp_mode}*')
for m in model_list[:]:
    print(f"Model:{m}")
    sim_cse = SentenceTransformer(m)
    
    count_parameters(sim_cse)

Model:output/Simcse_original_BERT_original_cls_MLPTrue
Total Trainable Params: 109,482,240
Model:output/Simcse_original_nreimers_albert_small_v2_cls_MLPTrue
Total Trainable Params: 12,274,176
Model:output/Simcse_original_nreimers_BERT_Mini_L_4_H_256_A_4_cls_MLPTrue
Total Trainable Params: 11,236,352
Model:output/Simcse_original_nreimers_MiniLM_L6_H384_uncased_cls_MLPTrue
Total Trainable Params: 22,861,056
Model:output/Simcse_original_bert_large_uncased_cls_MLPTrue
Total Trainable Params: 336,191,488
Model:output/Simcse_original_distilbert_base_cased_cls_MLPTrue
Total Trainable Params: 65,781,504
Model:output/Simcse_original_nreimers_TinyBERT_L_4_H_312_v2_cls_MLPTrue
Total Trainable Params: 14,447,904
Model:output/Simcse_original_roberta_base_cls_MLPTrue
Total Trainable Params: 125,236,224
Model:output/Simcse_original_nreimers_BERT_Tiny_L_2_H_128_A_2_cls_MLPTrue
Total Trainable Params: 4,402,432
Model:output/Simcse_original_roberta_large_cls_MLPTrue
Total Trainable Params: 356,409,344
M

# L2 Mono T = SimCSE(Wiki) S = small model

In [4]:
pool_mode = 'cls'
mlp_mode = True
model_list = glob(f'output/l2_mono/L2-T*')
# making-T-output-Simcse_original_roberta_large_cls_MLPTrue-S-bert-base-cased
for model in model_list:
    try:
        word_embedding_model = models.Transformer(model, max_seq_length=32)
        pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
        sim_cse = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    except:
        sim_cse = SentenceTransformer(model)
    
    params = {'task_path': 'SentEval/data/', 'usepytorch': True, 'kfold': 10}
    params['encoder'] = sim_cse.encode
    se = senteval.engine.SE(params, batcher, prepare)

    transfer_tasks = ['STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark', 'SICKRelatedness']
    results = se.eval(transfer_tasks)

    print(f"Model:{model}")
    spearman_val = 0
    for key in results.keys():
        print(key)
    #     print(results)
        if key not in ['STSBenchmark','SICKRelatedness']:
            result_temp = results[key]['all']['spearman']['all']
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
        else:
            result_temp = results[key]['test']['spearman'].correlation
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
    print(f"Avg:{(spearman_val/len(results.keys()))*100:.2f}")
    print(f"*"*50)
    print()

/workspace/m_sentence_embedding/SBERT/SentEval/senteval/sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
/workspace/m_sentence_embedding/SBERT/SentEval/senteval/sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


Model:output/l2_mono/L2-T-output-simcse_original-Simcse_original_roberta_large_cls_MLPTrue-S-nreimers-BERT-Tiny_L-2_H-128_A-2
STS12
Spearman:70.78
STS13
Spearman:78.98
STS14
Spearman:70.98
STS15
Spearman:78.58
STS16
Spearman:74.61
STSBenchmark
Spearman:73.42
SICKRelatedness
Spearman:67.43
Avg:73.54
**************************************************

Model:output/l2_mono/L2-T-output-simcse_original-Simcse_original_roberta_large_cls_MLPTrue-S-distilbert-base-cased
STS12
Spearman:72.64
STS13
Spearman:84.11
STS14
Spearman:75.86
STS15
Spearman:83.35
STS16
Spearman:79.10
STSBenchmark
Spearman:80.53
SICKRelatedness
Spearman:70.07
Avg:77.95
**************************************************

Model:output/l2_mono/L2-T-output-simcse_original-Simcse_original_roberta_large_cls_MLPTrue-S-nreimers-TinyBERT_L-6_H-768_v2
STS12
Spearman:72.28
STS13
Spearman:84.38
STS14
Spearman:76.77
STS15
Spearman:84.91
STS16
Spearman:79.89
STSBenchmark
Spearman:81.23
SICKRelatedness
Spearman:70.70
Avg:78.59
*********

# SimCSE+BSL+Distill Mono T = SimCSE(Wiki) S = small model

In [5]:
pool_mode = 'cls'
mlp_mode = True
model_list = glob(f'output/bsl_mono_distill/BSL_making-test-T-*-S*')
# making-T-output-Simcse_original_roberta_large_cls_MLPTrue-S-bert-base-cased
for model in model_list:
    print(f"Model:{model}")
#     word_embedding_model = models.Transformer(model, max_seq_length=32)
#     pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
#     sim_cse = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    sim_cse = SentenceTransformer(model)
    
    params = {'task_path': 'SentEval/data/', 'usepytorch': True, 'kfold': 10}
    params['encoder'] = sim_cse.encode
    se = senteval.engine.SE(params, batcher, prepare)

    transfer_tasks = ['STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark', 'SICKRelatedness']
    results = se.eval(transfer_tasks)

    
    spearman_val = 0
    for key in results.keys():
        print(key)
    #     print(results)
        if key not in ['STSBenchmark','SICKRelatedness']:
            result_temp = results[key]['all']['spearman']['all']
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
        else:
            result_temp = results[key]['test']['spearman'].correlation
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
    print(f"Avg:{(spearman_val/len(results.keys()))*100:.2f}")
    print(f"*"*50)
    print()

Model:output/bsl_mono_distill/BSL_making-test-T-princeton-nlp-unsup-simcse-roberta-large-S--Simcse_original_nreimers_BERT_Mini_L_4_H_256_A_4_cls_MLPTrue
STS12
Spearman:75.19
STS13
Spearman:81.98
STS14
Spearman:76.03
STS15
Spearman:83.63
STS16
Spearman:77.31
STSBenchmark
Spearman:79.76
SICKRelatedness
Spearman:70.69
Avg:77.80
**************************************************

Model:output/bsl_mono_distill/BSL_making-test-T-princeton-nlp-unsup-simcse-roberta-large-S--Simcse_original_nreimers_TinyBERT_L_4_H_312_v2_cls_MLPTrue
STS12
Spearman:75.63
STS13
Spearman:82.69
STS14
Spearman:76.38
STS15
Spearman:84.19
STS16
Spearman:78.99
STSBenchmark
Spearman:80.22
SICKRelatedness
Spearman:70.71
Avg:78.40
**************************************************

Model:output/bsl_mono_distill/BSL_making-test-T-princeton-nlp-unsup-simcse-roberta-large-S--Simcse_original_distilbert_base_cased_cls_MLPTrue
STS12
Spearman:77.73
STS13
Spearman:83.99
STS14
Spearman:77.27
STS15
Spearman:84.32
STS16
Spearman:79.

# BSL+Distill Mono T = SimCSE(Wiki) S = small model

In [15]:
pool_mode = 'cls'
mlp_mode = True
model_list = glob(f'output/bsl_mono_distill/BSL_making-T-*-S*')
# making-T-output-Simcse_original_roberta_large_cls_MLPTrue-S-bert-base-cased
for model in model_list:
    print(f"Model:{model}")
#     word_embedding_model = models.Transformer(model, max_seq_length=32)
#     pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
#     sim_cse = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    sim_cse = SentenceTransformer(model)
    
    params = {'task_path': 'SentEval/data/', 'usepytorch': True, 'kfold': 10}
    params['encoder'] = sim_cse.encode
    se = senteval.engine.SE(params, batcher, prepare)

    transfer_tasks = ['STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark', 'SICKRelatedness']
    results = se.eval(transfer_tasks)

    
    spearman_val = 0
    for key in results.keys():
        print(key)
    #     print(results)
        if key not in ['STSBenchmark','SICKRelatedness']:
            result_temp = results[key]['all']['spearman']['all']
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
        else:
            result_temp = results[key]['test']['spearman'].correlation
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
    print(f"Avg:{(spearman_val/len(results.keys()))*100:.2f}")
    print(f"*"*50)
    print()

Model:output/bsl_mono_distill/BSL_making-T-princeton-nlp-unsup-simcse-roberta-large-S-nreimers-BERT-Tiny_L-2_H-128_A-2
STS12
Spearman:73.63
STS13
Spearman:80.76
STS14
Spearman:74.63
STS15
Spearman:82.39
STS16
Spearman:76.08
STSBenchmark
Spearman:77.84
SICKRelatedness
Spearman:69.15
Avg:76.35
**************************************************

Model:output/bsl_mono_distill/BSL_making-T-princeton-nlp-unsup-simcse-roberta-large-S-nreimers-TinyBERT_L-4_H-312_v2
STS12
Spearman:75.42
STS13
Spearman:82.47
STS14
Spearman:76.60
STS15
Spearman:84.32
STS16
Spearman:78.96
STSBenchmark
Spearman:80.33
SICKRelatedness
Spearman:70.96
Avg:78.44
**************************************************

Model:output/bsl_mono_distill/BSL_making-T-princeton-nlp-unsup-simcse-roberta-large-S-distilbert-base-cased
STS12
Spearman:77.57
STS13
Spearman:83.73
STS14
Spearman:77.48
STS15
Spearman:84.21
STS16
Spearman:79.35
STSBenchmark
Spearman:81.38
SICKRelatedness
Spearman:71.28
Avg:79.29
******************************

# SKD distill T=Robert-Large (SimCSE-wiki1M)

In [9]:
pool_mode = 'cls'
mlp_mode = True
model_list = glob(f'output/skd/skd_mono-T*')
# making-T-output-Simcse_original_roberta_large_cls_MLPTrue-S-bert-base-cased
for model in model_list:
#     word_embedding_model = models.Transformer(model, max_seq_length=32)
#     pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
#     sim_cse = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    sim_cse = SentenceTransformer(model)
    
    params = {'task_path': 'SentEval/data/', 'usepytorch': True, 'kfold': 10}
    params['encoder'] = sim_cse.encode
    se = senteval.engine.SE(params, batcher, prepare)

    transfer_tasks = ['STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark', 'SICKRelatedness']
    results = se.eval(transfer_tasks)

    print(f"Model:{model}")
    spearman_val = 0
    for key in results.keys():
        print(key)
    #     print(results)
        if key not in ['STSBenchmark','SICKRelatedness']:
            result_temp = results[key]['all']['spearman']['all']
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
        else:
            result_temp = results[key]['test']['spearman'].correlation
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
    print(f"Avg:{(spearman_val/len(results.keys()))*100:.2f}")
    print(f"*"*50)
    print()

Model:output/skd/skd_mono-T-princeton-nlp-unsup-simcse-roberta-large-S-nreimers-MiniLM-L6-H384-uncased
STS12
Spearman:67.87
STS13
Spearman:78.04
STS14
Spearman:68.79
STS15
Spearman:77.13
STS16
Spearman:70.63
STSBenchmark
Spearman:72.21
SICKRelatedness
Spearman:67.86
Avg:71.79
**************************************************

Model:output/skd/skd_mono-T-princeton-nlp-unsup-simcse-roberta-large-S-distilbert-base-cased
STS12
Spearman:72.63
STS13
Spearman:80.98
STS14
Spearman:72.40
STS15
Spearman:79.42
STS16
Spearman:74.25
STSBenchmark
Spearman:76.16
SICKRelatedness
Spearman:68.79
Avg:74.95
**************************************************

Model:output/skd/skd_mono-T-princeton-nlp-unsup-simcse-roberta-large-S-nreimers-MiniLM-L3-H384-uncased
STS12
Spearman:69.34
STS13
Spearman:78.35
STS14
Spearman:70.17
STS15
Spearman:79.35
STS16
Spearman:71.64
STSBenchmark
Spearman:72.39
SICKRelatedness
Spearman:66.18
Avg:72.49
**************************************************

Model:output/skd/skd_mo

# BSL original

In [18]:
pool_mode = 'cls'
mlp_mode = True
model_list = glob(f'output/bsl_original/BSL_unsup_tuning-*')
for model in model_list:
    sim_cse = SentenceTransformer(model)
    print(f"Model:{model}")
#     word_embedding_model = models.Transformer(model, max_seq_length=32)
#     pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),pooling_mode=pool_mode)
#     sim_cse = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    
    params = {'task_path': 'SentEval/data/', 'usepytorch': True, 'kfold': 10}
    params['encoder'] = sim_cse.encode
    se = senteval.engine.SE(params, batcher, prepare)

    transfer_tasks = ['STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark', 'SICKRelatedness']
    results = se.eval(transfer_tasks)

    
    spearman_val = 0
    for key in results.keys():
        print(key)
    #     print(results)
        if key not in ['STSBenchmark','SICKRelatedness']:
            result_temp = results[key]['all']['spearman']['all']
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
        else:
            result_temp = results[key]['test']['spearman'].correlation
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
    print(f"Avg:{(spearman_val/len(results.keys()))*100:.2f}")
    print(f"*"*50)
    print()

Model:output/bsl_original/BSL_unsup_tuning-nreimers-TinyBERT_L-6_H-768_v2
STS12
Spearman:71.20
STS13
Spearman:68.56
STS14
Spearman:66.96
STS15
Spearman:79.78
STS16
Spearman:74.17
STSBenchmark
Spearman:74.68
SICKRelatedness
Spearman:67.92
Avg:71.89
**************************************************

Model:output/bsl_original/BSL_unsup_tuning-microsoft-MiniLM-L12-H384-uncased
STS12
Spearman:61.62
STS13
Spearman:59.90
STS14
Spearman:54.95
STS15
Spearman:72.29
STS16
Spearman:67.14
STSBenchmark
Spearman:67.57
SICKRelatedness
Spearman:64.45
Avg:63.99
**************************************************

Model:output/bsl_original/BSL_unsup_tuning-nreimers-TinyBERT_L-4_H-312_v2
STS12
Spearman:67.46
STS13
Spearman:59.80
STS14
Spearman:59.99
STS15
Spearman:77.18
STS16
Spearman:69.56
STSBenchmark
Spearman:70.50
SICKRelatedness
Spearman:67.14
Avg:67.38
**************************************************

Model:output/bsl_original/BSL_unsup_tuning-nreimers-BERT-Tiny_L-2_H-128_A-2
STS12
Spearman:65.57


# SimCSE (Supervised)

In [11]:
pool_mode = 'cls'
mlp_mode = True
model_list = glob(f'output/simcse_original/Simcse_supervised_original_*')
for model in model_list:
    sim_cse = SentenceTransformer(model)
    print(f"Model:{model}")
#     word_embedding_model = models.Transformer(model, max_seq_length=32)
#     pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),pooling_mode=pool_mode)
#     sim_cse = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    
    params = {'task_path': 'SentEval/data/', 'usepytorch': True, 'kfold': 10}
    params['encoder'] = sim_cse.encode
    se = senteval.engine.SE(params, batcher, prepare)

    transfer_tasks = ['STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark', 'SICKRelatedness']
    results = se.eval(transfer_tasks)

    
    spearman_val = 0
    for key in results.keys():
        print(key)
    #     print(results)
        if key not in ['STSBenchmark','SICKRelatedness']:
            result_temp = results[key]['all']['spearman']['all']
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
        else:
            result_temp = results[key]['test']['spearman'].correlation
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
    print(f"Avg:{(spearman_val/len(results.keys()))*100:.2f}")
    print(f"*"*50)
    print()

Model:output/simcse_original/Simcse_original_supervised_bert_base_uncased_cls_MLPTrue
STS12
Spearman:75.51
STS13
Spearman:83.84
STS14
Spearman:79.47
STS15
Spearman:85.50
STS16
Spearman:80.62
STSBenchmark
Spearman:83.77
SICKRelatedness
Spearman:80.20
Avg:81.27
**************************************************

Model:output/simcse_original/Simcse_original_supervised_roberta_base_cls_MLPTrue
STS12
Spearman:76.42
STS13
Spearman:84.47
STS14
Spearman:79.46
STS15
Spearman:85.33
STS16
Spearman:82.60
STSBenchmark
Spearman:84.96
SICKRelatedness
Spearman:80.47
Avg:81.96
**************************************************



# SimCSE (NLI) Without MLP

In [10]:
pool_mode = 'cls'
mlp_mode = True
model_list = glob(f'output/simcse_original/Simcse_original_*{pool_mode}*MLP{mlp_mode}_wikiFalse_nliTrue')
for model in model_list:
    sim_cse = SentenceTransformer(model)
    print(f"Model:{model}")
#     word_embedding_model = models.Transformer(model, max_seq_length=32)
#     pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),pooling_mode=pool_mode)
#     sim_cse = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    
    params = {'task_path': 'SentEval/data/', 'usepytorch': True, 'kfold': 10}
    params['encoder'] = sim_cse.encode
    se = senteval.engine.SE(params, batcher, prepare)

    transfer_tasks = ['STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark', 'SICKRelatedness']
    results = se.eval(transfer_tasks)

    
    spearman_val = 0
    for key in results.keys():
        print(key)
    #     print(results)
        if key not in ['STSBenchmark','SICKRelatedness']:
            result_temp = results[key]['all']['spearman']['all']
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
        else:
            result_temp = results[key]['test']['spearman'].correlation
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
    print(f"Avg:{(spearman_val/len(results.keys()))*100:.2f}")
    print(f"*"*50)
    print()

Model:output/simcse_original/Simcse_original_roberta_base_cls_MLPTrue_wikiFalse_nliTrue
STS12
Spearman:65.25
STS13
Spearman:79.64
STS14
Spearman:70.97
STS15
Spearman:80.51
STS16
Spearman:77.47
STSBenchmark
Spearman:76.97
SICKRelatedness
Spearman:68.11
Avg:74.13
**************************************************

Model:output/simcse_original/Simcse_original_nreimers_MiniLM_L6_H384_uncased_cls_MLPTrue_wikiFalse_nliTrue
STS12
Spearman:53.35
STS13
Spearman:67.36
STS14
Spearman:60.08
STS15
Spearman:69.37
STS16
Spearman:68.64
STSBenchmark
Spearman:62.45
SICKRelatedness
Spearman:64.85
Avg:63.73
**************************************************

Model:output/simcse_original/Simcse_original_nreimers_TinyBERT_L_6_H_768_v2_cls_MLPTrue_wikiFalse_nliTrue
STS12
Spearman:65.23
STS13
Spearman:69.03
STS14
Spearman:64.08
STS15
Spearman:76.66
STS16
Spearman:73.05
STSBenchmark
Spearman:70.89
SICKRelatedness
Spearman:70.71
Avg:69.95
**************************************************

Model:output/simcse_o

# SimCSE (Wiki1M) Without MLP

In [4]:
pool_mode = 'cls'
mlp_mode = True
model_list = glob(f'output/Simcse_original_*{pool_mode}*MLP{mlp_mode}*')
# model_list = ['output/Simcse_original_bert_base_uncased_cls_MLPTrue']
# model_list = glob(f'output/Simcse_original_*large*{pool_mode}*MLP{mlp_mode}*')
for model in model_list:
#     sim_cse = SentenceTransformer(model)
    word_embedding_model = models.Transformer(model, max_seq_length=32)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),pooling_mode=pool_mode)
    sim_cse = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    
    params = {'task_path': 'SentEval/data/', 'usepytorch': True, 'kfold': 10}
    params['encoder'] = sim_cse.encode
    se = senteval.engine.SE(params, batcher, prepare)

    transfer_tasks = ['STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark', 'SICKRelatedness']
    results = se.eval(transfer_tasks)

    print(f"Model:{model}")
    spearman_val = 0
    for key in results.keys():
        print(key)
    #     print(results)
        if key not in ['STSBenchmark','SICKRelatedness']:
            result_temp = results[key]['all']['spearman']['all']
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
        else:
            result_temp = results[key]['test']['spearman'].correlation
            spearman_val+=result_temp
            print(f"Spearman:{result_temp*100:.2f}")
    print(f"Avg:{(spearman_val/len(results.keys()))*100:.2f}")
    print(f"*"*50)
    print()

/workspace/m_sentence_embedding/SBERT/SentEval/senteval/sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
/workspace/m_sentence_embedding/SBERT/SentEval/senteval/sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


Model:output/Simcse_original_BERT_original_cls_MLPTrue
STS12
Spearman:68.64
STS13
Spearman:82.36
STS14
Spearman:74.30
STS15
Spearman:80.69
STS16
Spearman:78.71
STSBenchmark
Spearman:76.56
SICKRelatedness
Spearman:72.23
Avg:76.21
**************************************************

Model:output/Simcse_original_nreimers_albert_small_v2_cls_MLPTrue
STS12
Spearman:55.05
STS13
Spearman:69.71
STS14
Spearman:62.58
STS15
Spearman:72.92
STS16
Spearman:72.38
STSBenchmark
Spearman:67.77
SICKRelatedness
Spearman:66.13
Avg:66.65
**************************************************

Model:output/Simcse_original_nreimers_BERT_Mini_L_4_H_256_A_4_cls_MLPTrue
STS12
Spearman:56.55
STS13
Spearman:65.77
STS14
Spearman:59.55
STS15
Spearman:72.26
STS16
Spearman:70.23
STSBenchmark
Spearman:60.85
SICKRelatedness
Spearman:62.19
Avg:63.91
**************************************************

Model:output/Simcse_original_nreimers_MiniLM_L6_H384_uncased_cls_MLPTrue
STS12
Spearman:54.21
STS13
Spearman:65.43
STS14
Spearm